# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [3]:
import math
def InitialDataEqualMassConic(ecc,angle,initmass):
    orbitalangle=angle
    phi=np.array([orbitalangle,math.pi+orbitalangle])
    orbitalradius=100*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [72]:
import random,numpy as np
initdateqellipse=InitialDataEqualMassConic(0.1,0.,1.)
print(initdateqellipse)

0.1
(array([0.        , 3.14159265]), 0.0, array([100., 100.]), 0.1, array([1., 1.]))


In [88]:
def getxyuveqconic(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    if eccentricity<1 and eccentricity>0:
        #elliptical
        #orbital radius is the semi major axis. the separation between the stars is coordsep*2. the virialized
        #factor in E is 2*(orbitalraidus*2)
        E=-Gconstant*masses[0]*masses[1]/2/(2*orbitalradius[0]) #1/2*mv^2-GMm/r #orbitalradius is correct here r1+r2
        coordsep=orbitalradius[0]*(1.+eccentricity) #starts at apastron
        starsep=2.*coordsep
        Eapastron=Gconstant*masses[0]*masses[1]/(starsep)
        Fapastron=Eapastron/(starsep)
        print("E,F",E,Fapastron,2*orbitalradius,starsep)
        x0=coordsep*cosphi
        y0=coordsep*sinphi
        v=np.zeros(2)
        v=np.sqrt(2.*(E+Eapastron)/masses) #only works for equal mass system
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        for i in np.arange(2):
            a[i]=Gconstant*masses[(i+1)%2]/(r0[i]**2)
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [89]:
xyuvaeqconic=getxyuveqconic(initdateqellipse)
print(xyuvaeqconic)

initdat [100. 100.] [0.         3.14159265] [ 1. -1.] [0.0000000e+00 1.2246468e-16]
coordsep [100. 100.]
zero
pi
[ 100. -100.]
[0. 0.]
0.1
E,F -0.0025 2.0661157024793383e-05 [200. 200.] 220.00000000000003
(array([1., 1.]), array([ 110., -110.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0.06396021, -0.06396021]), array([0., 0.]), array([-2.0661157e-05,  2.0661157e-05]), array([-0., -0.]), array([0., 0.]))


In [90]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                #print(rreljk)
                axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [91]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqconic
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 110. -110.]
[ 110. -110.] [ 0.06396021 -0.06396021] [-2.0661157e-05  2.0661157e-05] [ 0.06396021 -0.06396021] [-2.0661157e-05  2.0661157e-05] [0. 0.]
[ 109.99997934 -109.99997934] [ 0.12792043 -0.12792043] [-4.1322314e-05  4.1322314e-05] [ 0.06396021 -0.06396021] [-2.06611465e-05  2.06611465e-05] [-1.20135579e-08  1.20135579e-08]
[ 109.99993802 -109.99993802] [ 0.19188064 -0.19188064] [-6.19834606e-05  6.19834606e-05] [ 0.0639602 -0.0639602] [-2.06611229e-05  2.06611229e-05] [-2.40270929e-08  2.40270929e-08]
[ 109.99987603 -109.99987603] [ 0.25584085 -0.25584085] [-8.26445835e-05  8.26445835e-05] [ 0.06396018 -0.06396018] [-2.0661086e-05  2.0661086e-05] [-3.60405885e-08  3.60405885e-08]
[ 109.99979339 -109.99979339] [ 0.31980103 -0.31980103] [-0.00010331  0.00010331] [ 0.06396014 -0.06396014] [-2.06610359e-05  2.06610359e-05] [-4.80540264e-08  4.80540264e-08]
[ 109.99969008 -109.99969008] [ 0.38376117 -0.38376117] [-0.00012397  0.00012397] [ 0.06396009 -0.06396009] [-2.06609727e-05  

[ 109.66026928 -109.66026928] [ 11.62911143 -11.62911143] [-0.00374739  0.00374739] [ 0.06376533 -0.06376533] [-2.04459094e-05  2.04459094e-05] [-2.15625897e-06  2.15625897e-06]
[ 109.65652189 -109.65652189] [ 11.69287676 -11.69287676] [-0.00376784  0.00376784] [ 0.06376317 -0.06376317] [-2.0443546e-05  2.0443546e-05] [-2.16797092e-06  2.16797092e-06]
[ 109.65275405 -109.65275405] [ 11.75663994 -11.75663994] [-0.00378828  0.00378828] [ 0.06376101 -0.06376101] [-2.044117e-05  2.044117e-05] [-2.17967958e-06  2.17967958e-06]
[ 109.64896577 -109.64896577] [ 11.82040094 -11.82040094] [-0.00380872  0.00380872] [ 0.06375883 -0.06375883] [-2.04387813e-05  2.04387813e-05] [-2.19138493e-06  2.19138493e-06]
[ 109.64515705 -109.64515705] [ 11.88415977 -11.88415977] [-0.00382916  0.00382916] [ 0.06375664 -0.06375664] [-2.04363801e-05  2.04363801e-05] [-2.20308696e-06  2.20308696e-06]
[ 109.64132789 -109.64132789] [ 11.9479164 -11.9479164] [-0.0038496  0.0038496] [ 0.06375443 -0.06375443] [-2.043396

[ 108.7955433 -108.7955433] [ 21.85966499 -21.85966499] [-0.00700033  0.00700033] [ 0.06327002 -0.06327002] [-1.99080791e-05  1.99080791e-05] [-3.98818209e-06  3.98818209e-06]
[ 108.78854297 -108.78854297] [ 21.92293501 -21.92293501] [-0.00702024  0.00702024] [ 0.06326603 -0.06326603] [-1.9903787e-05  1.9903787e-05] [-3.99915385e-06  3.99915385e-06]
[ 108.78152273 -108.78152273] [ 21.98620104 -21.98620104] [-0.00704014  0.00704014] [ 0.06326203 -0.06326203] [-1.98994837e-05  1.98994837e-05] [-4.01011977e-06  4.01011977e-06]
[ 108.77448259 -108.77448259] [ 22.04946307 -22.04946307] [-0.00706004  0.00706004] [ 0.06325802 -0.06325802] [-1.98951691e-05  1.98951691e-05] [-4.02107984e-06  4.02107984e-06]
[ 108.76742255 -108.76742255] [ 22.1127211 -22.1127211] [-0.00707994  0.00707994] [ 0.063254 -0.063254] [-1.98908434e-05  1.98908434e-05] [-4.03203404e-06  4.03203404e-06]
[ 108.76034261 -108.76034261] [ 22.1759751 -22.1759751] [-0.00709983  0.00709983] [ 0.06324997 -0.06324997] [-1.98865065

[ 107.99770259 -107.99770259] [ 28.16517532 -28.16517532] [-0.00896812  0.00896812] [ 0.06281789 -0.06281789] [-1.94252024e-05  1.94252024e-05] [-5.05426173e-06  5.05426173e-06]
[ 107.98873447 -107.98873447] [ 28.22799321 -28.22799321] [-0.00898755  0.00898755] [ 0.06281284 -0.06281284] [-1.94198463e-05  1.94198463e-05] [-5.06458344e-06  5.06458344e-06]
[ 107.97974692 -107.97974692] [ 28.29080604 -28.29080604] [-0.00900697  0.00900697] [ 0.06280777 -0.06280777] [-1.94144802e-05  1.94144802e-05] [-5.07489802e-06  5.07489802e-06]
[ 107.97073996 -107.97073996] [ 28.35361381 -28.35361381] [-0.00902638  0.00902638] [ 0.0628027 -0.0628027] [-1.9409104e-05  1.9409104e-05] [-5.08520546e-06  5.08520546e-06]
[ 107.96171357 -107.96171357] [ 28.41641651 -28.41641651] [-0.00904579  0.00904579] [ 0.06279761 -0.06279761] [-1.94037179e-05  1.94037179e-05] [-5.09550573e-06  5.09550573e-06]
[ 107.95266778 -107.95266778] [ 28.47921412 -28.47921412] [-0.00906519  0.00906519] [ 0.06279252 -0.06279252] [-1.

[ 106.78441602 -106.78441602] [ 35.66439663 -35.66439663] [-0.0112582  0.0112582] [ 0.06213978 -0.06213978] [-1.87147084e-05  1.87147084e-05] [-6.23888429e-06  6.23888429e-06]
[ 106.77315782 -106.77315782] [ 35.7265364 -35.7265364] [-0.01127691  0.01127691] [ 0.06213354 -0.06213354] [-1.87082469e-05  1.87082469e-05] [-6.24827445e-06  6.24827445e-06]
[ 106.76188091 -106.76188091] [ 35.78866994 -35.78866994] [-0.01129562  0.01129562] [ 0.06212729 -0.06212729] [-1.87017771e-05  1.87017771e-05] [-6.25765628e-06  6.25765628e-06]
[ 106.75058529 -106.75058529] [ 35.85079723 -35.85079723] [-0.01131432  0.01131432] [ 0.06212103 -0.06212103] [-1.8695299e-05  1.8695299e-05] [-6.26702977e-06  6.26702977e-06]
[ 106.73927096 -106.73927096] [ 35.91291826 -35.91291826] [-0.01133302  0.01133302] [ 0.06211476 -0.06211476] [-1.86888126e-05  1.86888126e-05] [-6.27639491e-06  6.27639491e-06]
[ 106.72793795 -106.72793795] [ 35.97503302 -35.97503302] [-0.01135171  0.01135171] [ 0.06210849 -0.06210849] [-1.86

...

[-172.01196752  172.01196752] [ 288.39896415 -288.39896415] [-0.03054268  0.03054268] [ 0.01026551 -0.01026551] [ 1.13572152e-06 -1.13572152e-06] [-1.9044455e-06  1.9044455e-06]
[-172.04251019  172.04251019] [ 288.40922967 -288.40922967] [-0.03054154  0.03054154] [ 0.01026361 -0.01026361] [ 1.13567498e-06 -1.13567498e-06] [-1.90409709e-06  1.90409709e-06]
[-172.07305173  172.07305173] [ 288.41949328 -288.41949328] [-0.0305404  0.0305404] [ 0.01026171 -0.01026171] [ 1.13562843e-06 -1.13562843e-06] [-1.90374879e-06  1.90374879e-06]
[-172.10359213  172.10359213] [ 288.42975498 -288.42975498] [-0.03053927  0.03053927] [ 0.0102598 -0.0102598] [ 1.13558186e-06 -1.13558186e-06] [-1.90340058e-06  1.90340058e-06]
[-172.1341314  172.1341314] [ 288.44001478 -288.44001478] [-0.03053813  0.03053813] [ 0.0102579 -0.0102579] [ 1.13553528e-06 -1.13553528e-06] [-1.90305246e-06  1.90305246e-06]
[-172.16466954  172.16466954] [ 288.45027268 -288.45027268] [-0.030537  0.030537] [ 0.010256 -0.010256] [ 1.13

[-176.36806153  176.36806153] [ 289.84776154 -289.84776154] [-0.03038075  0.03038075] [ 0.00999667 -0.00999667] [ 1.12893001e-06 -1.12893001e-06] [-1.85556796e-06  1.85556796e-06]
[-176.39844228  176.39844228] [ 289.85775821 -289.85775821] [-0.03037962  0.03037962] [ 0.00999482 -0.00999482] [ 1.12888159e-06 -1.12888159e-06] [-1.85523274e-06  1.85523274e-06]
[-176.4288219  176.4288219] [ 289.86775303 -289.86775303] [-0.03037849  0.03037849] [ 0.00999296 -0.00999296] [ 1.12883316e-06 -1.12883316e-06] [-1.85489761e-06  1.85489761e-06]
[-176.45920038  176.45920038] [ 289.87774599 -289.87774599] [-0.03037736  0.03037736] [ 0.00999111 -0.00999111] [ 1.12878472e-06 -1.12878472e-06] [-1.85456257e-06  1.85456257e-06]
[-176.48957774  176.48957774] [ 289.88773709 -289.88773709] [-0.03037623  0.03037623] [ 0.00998925 -0.00998925] [ 1.12873626e-06 -1.12873626e-06] [-1.85422762e-06  1.85422762e-06]
[-176.51995397  176.51995397] [ 289.89772634 -289.89772634] [-0.0303751  0.0303751] [ 0.0099874 -0.009

[-181.1844518  181.1844518] [ 291.41414256 -291.41414256] [-0.03020186  0.03020186] [ 0.00970579 -0.00970579] [ 1.1210861e-06 -1.1210861e-06] [-1.80337731e-06  1.80337731e-06]
[-181.21465366  181.21465366] [ 291.42384835 -291.42384835] [-0.03020074  0.03020074] [ 0.00970398 -0.00970398] [ 1.12103589e-06 -1.12103589e-06] [-1.803056e-06  1.803056e-06]
[-181.2448544  181.2448544] [ 291.43355234 -291.43355234] [-0.03019962  0.03019962] [ 0.00970218 -0.00970218] [ 1.12098567e-06 -1.12098567e-06] [-1.80273478e-06  1.80273478e-06]
[-181.27505402  181.27505402] [ 291.44325452 -291.44325452] [-0.0301985  0.0301985] [ 0.00970038 -0.00970038] [ 1.12093544e-06 -1.12093544e-06] [-1.80241364e-06  1.80241364e-06]
[-181.30525252  181.30525252] [ 291.4529549 -291.4529549] [-0.03019738  0.03019738] [ 0.00969858 -0.00969858] [ 1.1208852e-06 -1.1208852e-06] [-1.80209258e-06  1.80209258e-06]
[-181.3354499  181.3354499] [ 291.46265347 -291.46265347] [-0.03019626  0.03019626] [ 0.00969677 -0.00969677] [ 1.12

[-184.83075257  184.83075257] [ 292.57554225 -292.57554225] [-0.03006658  0.03006658] [ 0.00948989 -0.00948989] [ 1.11493884e-06 -1.11493884e-06] [-1.76510846e-06  1.76510846e-06]
[-184.86081915  184.86081915] [ 292.58503213 -292.58503213] [-0.03006546  0.03006546] [ 0.00948812 -0.00948812] [ 1.11488746e-06 -1.11488746e-06] [-1.76479724e-06  1.76479724e-06]
[-184.89088461  184.89088461] [ 292.59452026 -292.59452026] [-0.03006435  0.03006435] [ 0.00948636 -0.00948636] [ 1.11483607e-06 -1.11483607e-06] [-1.7644861e-06  1.7644861e-06]
[-184.92094896  184.92094896] [ 292.60400661 -292.60400661] [-0.03006323  0.03006323] [ 0.00948459 -0.00948459] [ 1.11478467e-06 -1.11478467e-06] [-1.76417504e-06  1.76417504e-06]
[-184.95101219  184.95101219] [ 292.61349121 -292.61349121] [-0.03006212  0.03006212] [ 0.00948283 -0.00948283] [ 1.11473326e-06 -1.11473326e-06] [-1.76386407e-06  1.76386407e-06]
[-184.98107431  184.98107431] [ 292.62297404 -292.62297404] [-0.030061  0.030061] [ 0.00948107 -0.0094

[-188.90959324  188.90959324] [ 293.85008984 -293.85008984] [-0.02991542  0.02991542] [ 0.00925266 -0.00925266] [ 1.10787191e-06 -1.10787191e-06] [-1.72352057e-06  1.72352057e-06]
[-188.93950866  188.93950866] [ 293.8593425 -293.8593425] [-0.02991431  0.02991431] [ 0.00925093 -0.00925093] [ 1.10781939e-06 -1.10781939e-06] [-1.7232202e-06  1.7232202e-06]
[-188.96942297  188.96942297] [ 293.86859343 -293.86859343] [-0.02991321  0.02991321] [ 0.00924921 -0.00924921] [ 1.10776686e-06 -1.10776686e-06] [-1.72291991e-06  1.72291991e-06]
[-188.99933618  188.99933618] [ 293.87784265 -293.87784265] [-0.0299121  0.0299121] [ 0.00924749 -0.00924749] [ 1.10771432e-06 -1.10771432e-06] [-1.7226197e-06  1.7226197e-06]
[-189.02924828  189.02924828] [ 293.88709013 -293.88709013] [-0.02991099  0.02991099] [ 0.00924576 -0.00924576] [ 1.10766178e-06 -1.10766178e-06] [-1.72231956e-06  1.72231956e-06]
[-189.05915927  189.05915927] [ 293.8963359 -293.8963359] [-0.02990988  0.02990988] [ 0.00924404 -0.00924404

In [92]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Equal Mass System, Ellipse", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

It's a parabolic orbit, when it should be an elliptical orbit. Clearly the energy (initial velocity, with internally consistent acceleration and position but not eccentricity) is too high and it is unbound. 